# 00 라이브러리 로딩 #

In [1]:
import pandas as pd
import numpy as np
import regex
import gensim
from PyKomoran import *
komoran = Komoran("STABLE")
from tqdm import tqdm
tqdm.pandas()
import pickle

import seaborn
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
plt.rc('font', family='Malgun Gothic')

C:\Users\Administrator\Anaconda3\lib\site-packages\tqdm\std.py:656: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# 01 학습 데이터  #

## 데이터 전처리 ##

### 천만상상오아시스 파일 확인 ##

In [2]:
train_df = pd.read_csv('train.csv',sep='\t')
train_df

,글번호,제목,작성일,분류,내용,부서의견
0,150587,횡단보도 사각지대 반사경,2012-10-30,교통,운전을 하다보면 횡단보도 주변에 적치물들 때문에 사각지대가 형성되어 사람이 보이지 ...,"좋은 제안 주신 whatsmatter 님께 감사드리며, 제안을 검토한 결과를 다음과..."
1,150594,자전거 횡단보도 내려서 끌고가기,2012-10-30,교통,자전거 횡단보도와 자전거 표지를 보면 마치 자전거를 타고 건너도 되는 것처럼 보이고...,- 말씀하신 제안에는 충분히 공감합니다. 다만 교통안전표지(노면표시)는 도로교통법상...
2,150597,범죄예방을 위한 투도어현관보조문 설치,2012-10-30,주택,- 주민을 위하여 노력하시는 직원여러분께 감사드립니다 - 아파트및 다세대 등 홀로 ...,"000님 안녕하세요? 000님께서 공동주택(아파트, 다세대, 연립)의 현관문에 1인..."
3,150659,각각 집마다 간이소화기를 만들어 배치하는 것을 의무로 하였으면 좋겠습니다.,2012-10-31,안전,어제 어린 남매 두명이 집에 전자렌지로 인해 불이나서중태에 빠졌다는 기사를 보았습니...,기시행중임 -소방시설설치유지 및 안전관리에 관한 법률 제8조제1항 및 서울특별시 주...
4,150660,예식장내 부대시설인 음식점의 횡포를...,2012-10-31,경제,자녀의 결혼등으로 인한 예식이나 회갑 칠순등으로 인하여 웨딩홀이나 호텔등을 이용시에...,안녕하십니까? 우리시정에 큰 관심을 보내주신 데 대하여 깊은 감사드립니다. dski...
...,...,...,...,...,...,...
10672,179106,지하철 임산부 지정석 관련,2017-08-31,교통,안녕하세요 수고하십니다.다름이 아니오라 지하철 임산부지정석 관련 건의 드립니다.현...,김혁님 안녕하십니까? 제안 참여에 감사드리며 “지하철 임산부 지정석”건에 대하여 답...
10673,179107,채동선 작곡가의 서울 집을 보존해 주세요.,2017-08-31,문화,서울 성북동의 옛날짜장면집 뒷 골목에 한옥집들 사이로 고풍스러운 2층 적산가옥이 보...,NaN
10674,179108,한강및 지천에서의 안전,2017-08-31,환경,시민의 안전과 편의를 위하여 불철주야 수고 하심에 감사드립니다.저는 주로 한강및 한...,NaN
10675,179109,세빛섬 이전 가능한가 ?,2017-08-31,환경,안녕하세요 ?최근에 노들섬이 2018년 말 경 개발된다는 기사를 본것 같습니다.예전...,NaN


### 컬럼 확인 ###

In [3]:
train_df.columns

Index(['글번호', '제목', '작성일', '분류', '내용', '부서의견'], dtype='object')

In [4]:
# 민원 분류별 개수 #
train_df['분류'].value_counts()

환경    2525
교통    2160
안전    1207
문화    1110
건강     930
주택     817
복지     681
경제     595
여성     504
세금     147
Name: 분류, dtype: int64

In [9]:
# 연도별 민원 개수 #
train_df['작성일'] = pd.to_datetime(train_df['작성일'])
train_df['작성일'].dt.year.value_counts()

2013    3540
2014    1991
2015    1432
2016    1372
2017     979
2012     558
Name: 작성일, dtype: int64

### 데이터 핸들링 ###

In [5]:
# '내용' 없는 행 삭제 #
train_df = train_df.dropna(subset=['내용'])

In [6]:
# '내용' 중복 행 삭제 #
train_df = train_df.drop_duplicates(subset='내용')

In [12]:
# 전처리 후 민원 분류 개수 및 연도별 민원 개수 재확인 #
print(train_df['분류'].value_counts())
print(train_df['작성일'].dt.year.value_counts())

환경    2391
교통    1980
안전    1097
문화    1060
건강     864
주택     708
복지     629
경제     541
여성     471
세금     131
Name: 분류, dtype: int64
2013    3540
2014    1991
2015    1432
2016    1372
2017     979
2012     558
Name: 작성일, dtype: int64


In [13]:
# 제목 + 내용 통합 #
train_df['contents'] = train_df.apply(lambda x:x['제목']+"\n"+x['내용'],axis=1)
train_df['contents']

0        횡단보도 사각지대 반사경\n운전을 하다보면 횡단보도 주변에 적치물들 때문에 사각지대...
1        자전거 횡단보도 내려서 끌고가기\n자전거 횡단보도와 자전거 표지를 보면 마치 자전거...
2        범죄예방을 위한 투도어현관보조문 설치\n- 주민을 위하여 노력하시는 직원여러분께 감...
3        각각 집마다 간이소화기를 만들어 배치하는 것을 의무로 하였으면 좋겠습니다.\n어제 ...
4        예식장내 부대시설인 음식점의 횡포를...\n자녀의 결혼등으로 인한 예식이나 회갑 칠...
                               ...                        
10672    지하철 임산부 지정석 관련\n안녕하세요  수고하십니다.다름이 아니오라 지하철 임산부...
10673    채동선 작곡가의 서울 집을 보존해 주세요.\n서울 성북동의 옛날짜장면집 뒷 골목에 ...
10674    한강및 지천에서의 안전\n시민의 안전과 편의를 위하여 불철주야 수고 하심에 감사드립...
10675    세빛섬 이전 가능한가 ?\n안녕하세요 ?최근에 노들섬이 2018년 말 경 개발된다는...
10676    한강 시민공원 주차장에 태양광 패널로 지붕을 설치하면 좋겠습니다.\n한강시민공원에서...
Name: contents, Length: 9872, dtype: object

## 형태소 분석 ##

### 사용자 사전 추가 ###

In [14]:
komoran.set_user_dic('dic.txt')

In [15]:
komoran.get_list('강변북로 교통혼잡')

[강변북로/NNP, 교통/NNG, 혼잡/NNG]

In [17]:
#특수문자 제거#
def cleanText(readData):
    #텍스트에 포함되어 있는 특수 문자 제거
    text = re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\’\“\”\·\n\r\t■◇◆▶;\xa0]', '', readData).strip()
    return text

#명사, 형용사, 관형사, 동사, 부사, 관형사#
def morp(strings):
   return [w.get_first()+'다' if w.get_second() in ['VV','VA'] else w.get_first() for w in komoran.get_list(cleanText(strings)) if w.get_second() in ['NNP','NNG','MAG','VA','VV','MM']]

In [18]:
morp('혼잡스러운 강변북로를 달리다')

['혼잡', '강변북로', '달리다']

### 토크나이징 ###

In [19]:
# 특정 품사 토크나이징
train_df['tokens'] = train_df['contents'].progress_map(lambda x:morp(x))

100%|██████████| 9872/9872 [17:39<00:00,  9.32it/s]


In [20]:
# 모든 품사 토크나이징
train_df['all_tokens'] = train_df['contents'].progress_map(lambda x:komoran.get_morphes_by_tags(cleanText(x)))

100%|██████████| 9872/9872 [04:59<00:00, 32.96it/s]


### 불용어(stopwords) 제거 ###

In [21]:
# 불용어 리스트 만들기
stopwords = ['아래', '상상', '제안', '까지', '닷컴', '포털', '사이트', '천만', '오아시스', '이벤트', '접수','서울시','서울','특별시',
             '천만상상','파일','첨부','응모','슬로건','공모','공모전','응모전','신청','경우','때문','정도','사항',
                   '해당','겁니다','이것','저것','그것','돋움','신명', '태명', '한컴', '돋움',
                   '동안','거기','저기','여기','대부분','누구','무엇','고딕','만큼','굴림','감사','건지','텐데',
                   '안녕','안녕하세요','이번','걸로','수고','겁니까','그간','그건','그때','글쓴이','누가','니다','다면',
                   '뭔가','상상오아시스','하다','이다','되다','같다','궁','자체','서체','정','서','이','을','있다','없다', '체','관련',
                   '생각', '현재', '진행', '사람', '마음', '남산', '내용', '현실','음','막','김','변','조',
                   '오','참','동','지금','주변','대상','부분','요즘','하루','마련','세대','시간','이상','행위',
                   '활동','구분','사실','과정','모습','기간','선정','단지','자신','발생','지역','기대','마련',
                   '장소','모두','부탁','제공','이용','해주','당시','최근','민원','문제','문제점','현황','개선','방안',
                   '문의','답변','일동','요청','담당자','직원','방법','사용','활용','확인','방식','예전','안녕하십니까',
                   '이하','바로 가기','바로가기','제가']
                   
# 기존 불용어 리스트에 추가
stopwords.append('홈페이지')
stopwords_set = set(stopwords)

In [22]:
# 불용어 제거
train_df['tokens'] = train_df['tokens'].map(lambda x:[w for w in x if not w in stopwords_set])
train_df['all_tokens'] = train_df['all_tokens'].map(lambda x:[w for w in x if not w in stopwords_set])

In [23]:
train_df['tokens']

0        [횡단보도, 사각지대, 반사경, 운전, 횡단보도, 적치물, 사각지대, 형성, 보이다...
1        [자전거, 횡단보도, 내리다, 끌다, 자전거, 횡단보도, 자전거, 표지, 보다, 마...
2        [범죄, 예방, 위하다, 투, 도어, 현관, 보조, 문, 설치, 주민, 위하다, 노...
3        [각각, 집, 간이, 소화기, 만들다, 배치, 의무, 좋다, 어제, 어리다, 남매,...
4        [예식장, 부대시설, 음식점, 횡포, 자녀, 결혼, 인하다, 예식, 회갑, 인하다,...
                               ...                        
10672    [지하철, 임산부, 지정석, 다르다, 지하철, 임산부, 지정석, 건의, 드리다, 현...
10673    [동선, 작곡가, 집, 보존, 주세, 요, 성북동, 옛날, 짜장면, 집, 뒷, 골목...
10674    [한강, 지천, 안전, 시민, 안전, 편의, 위하다, 불철주야, 하심, 드리다, 주...
10675    [세빛섬, 이전, 노들섬, 말, 경, 개발, 기사, 보다, 오페라, 하우스, 짓다,...
10676    [한강, 시민공원, 주차장, 태양광, 패널, 지붕, 설치, 좋다, 강, 시민공원, ...
Name: tokens, Length: 9872, dtype: object

In [24]:
train_df

,글번호,제목,작성일,분류,내용,부서의견,contents,tokens,all_tokens
0,150587,횡단보도 사각지대 반사경,2012-10-30,교통,운전을 하다보면 횡단보도 주변에 적치물들 때문에 사각지대가 형성되어 사람이 보이지 ...,"좋은 제안 주신 whatsmatter 님께 감사드리며, 제안을 검토한 결과를 다음과...",횡단보도 사각지대 반사경\n운전을 하다보면 횡단보도 주변에 적치물들 때문에 사각지대...,"[횡단보도, 사각지대, 반사경, 운전, 횡단보도, 적치물, 사각지대, 형성, 보이다...","[횡단보도, 사각지대, 반사경, 운전, 하, 다, 보, 면, 횡단보도, 에, 적치물..."
1,150594,자전거 횡단보도 내려서 끌고가기,2012-10-30,교통,자전거 횡단보도와 자전거 표지를 보면 마치 자전거를 타고 건너도 되는 것처럼 보이고...,- 말씀하신 제안에는 충분히 공감합니다. 다만 교통안전표지(노면표시)는 도로교통법상...,자전거 횡단보도 내려서 끌고가기\n자전거 횡단보도와 자전거 표지를 보면 마치 자전거...,"[자전거, 횡단보도, 내리다, 끌다, 자전거, 횡단보도, 자전거, 표지, 보다, 마...","[자전거, 횡단보도, 내리, 어서, 끌, 고, 가, 기, 자전거, 횡단보도, 와, ..."
2,150597,범죄예방을 위한 투도어현관보조문 설치,2012-10-30,주택,- 주민을 위하여 노력하시는 직원여러분께 감사드립니다 - 아파트및 다세대 등 홀로 ...,"000님 안녕하세요? 000님께서 공동주택(아파트, 다세대, 연립)의 현관문에 1인...",범죄예방을 위한 투도어현관보조문 설치\n- 주민을 위하여 노력하시는 직원여러분께 감...,"[범죄, 예방, 위하다, 투, 도어, 현관, 보조, 문, 설치, 주민, 위하다, 노...","[범죄, 예방, 위하, ㄴ, 투, 도어, 현관, 보조, 문, 설치, 주민, 위하, ..."
3,150659,각각 집마다 간이소화기를 만들어 배치하는 것을 의무로 하였으면 좋겠습니다.,2012-10-31,안전,어제 어린 남매 두명이 집에 전자렌지로 인해 불이나서중태에 빠졌다는 기사를 보았습니...,기시행중임 -소방시설설치유지 및 안전관리에 관한 법률 제8조제1항 및 서울특별시 주...,각각 집마다 간이소화기를 만들어 배치하는 것을 의무로 하였으면 좋겠습니다.\n어제 ...,"[각각, 집, 간이, 소화기, 만들다, 배치, 의무, 좋다, 어제, 어리다, 남매,...","[각각, 집, 마다, 간이, 소화기, 를, 만들, 어, 배치, 하, 는, 것, 의무..."
4,150660,예식장내 부대시설인 음식점의 횡포를...,2012-10-31,경제,자녀의 결혼등으로 인한 예식이나 회갑 칠순등으로 인하여 웨딩홀이나 호텔등을 이용시에...,안녕하십니까? 우리시정에 큰 관심을 보내주신 데 대하여 깊은 감사드립니다. dski...,예식장내 부대시설인 음식점의 횡포를...\n자녀의 결혼등으로 인한 예식이나 회갑 칠...,"[예식장, 부대시설, 음식점, 횡포, 자녀, 결혼, 인하다, 예식, 회갑, 인하다,...","[예식장, 내, 부대시설, ㄴ, 음식점, 의, 횡포, 를, 자녀, 의, 결혼, 등,..."
...,...,...,...,...,...,...,...,...,...
10672,179106,지하철 임산부 지정석 관련,2017-08-31,교통,안녕하세요 수고하십니다.다름이 아니오라 지하철 임산부지정석 관련 건의 드립니다.현...,김혁님 안녕하십니까? 제안 참여에 감사드리며 “지하철 임산부 지정석”건에 대하여 답...,지하철 임산부 지정석 관련\n안녕하세요 수고하십니다.다름이 아니오라 지하철 임산부...,"[지하철, 임산부, 지정석, 다르다, 지하철, 임산부, 지정석, 건의, 드리다, 현...","[지하철, 임산부, 지정석, 하, 시, 어요, 하, 시, ㅂ니다, 다르, ㅁ, 아니..."
10673,179107,채동선 작곡가의 서울 집을 보존해 주세요.,2017-08-31,문화,서울 성북동의 옛날짜장면집 뒷 골목에 한옥집들 사이로 고풍스러운 2층 적산가옥이 보...,NaN,채동선 작곡가의 서울 집을 보존해 주세요.\n서울 성북동의 옛날짜장면집 뒷 골목에 ...,"[동선, 작곡가, 집, 보존, 주세, 요, 성북동, 옛날, 짜장면, 집, 뒷, 골목...","[채, 동선, 작곡가, 의, 집, 보존, 하, 아, 주세, 요, 성북동, 의, 옛날..."
10674,179108,한강및 지천에서의 안전,2017-08-31,환경,시민의 안전과 편의를 위하여 불철주야 수고 하심에 감사드립니다.저는 주로 한강및 한...,NaN,한강및 지천에서의 안전\n시민의 안전과 편의를 위하여 불철주야 수고 하심에 감사드립...,"[한강, 지천, 안전, 시민, 안전, 편의, 위하다, 불철주야, 하심, 드리다, 주...","[한강, 및, 지천, 에서, 의, 안전, 시민, 의, 안전, 과, 편의, 를, 위하..."
10675,179109,세빛섬 이전 가능한가 ?,2017-08-31,환경,안녕하세요 ?최근에 노들섬이 2018년 말 경 개발된다는 기사를 본것 같습니다.예전...,NaN,세빛섬 이전 가능한가 ?\n안녕하세요 ?최근에 노들섬이 2018년 말 경 개발된다는...,"[세빛섬, 이전, 노들섬, 말, 경, 개발, 기사, 보다, 오페라, 하우스, 짓다,...","[세빛섬, 이전, 가능, 하, ㄴ가, 하, 시, 어요, 에, 노들섬, 2018, 년..."


In [25]:
# 토크나이징 후에도 중복된 행 제거
train_df["res"] = train_df["tokens"].astype(str) #list to string
train_df = train_df.drop_duplicates(subset="res")
train_df = train_df.drop('res',1)
train_df = train_df.reset_index(drop=True) # index 리셋

In [26]:
train_df

,글번호,제목,작성일,분류,내용,부서의견,contents,tokens,all_tokens
0,150587,횡단보도 사각지대 반사경,2012-10-30,교통,운전을 하다보면 횡단보도 주변에 적치물들 때문에 사각지대가 형성되어 사람이 보이지 ...,"좋은 제안 주신 whatsmatter 님께 감사드리며, 제안을 검토한 결과를 다음과...",횡단보도 사각지대 반사경\n운전을 하다보면 횡단보도 주변에 적치물들 때문에 사각지대...,"[횡단보도, 사각지대, 반사경, 운전, 횡단보도, 적치물, 사각지대, 형성, 보이다...","[횡단보도, 사각지대, 반사경, 운전, 하, 다, 보, 면, 횡단보도, 에, 적치물..."
1,150594,자전거 횡단보도 내려서 끌고가기,2012-10-30,교통,자전거 횡단보도와 자전거 표지를 보면 마치 자전거를 타고 건너도 되는 것처럼 보이고...,- 말씀하신 제안에는 충분히 공감합니다. 다만 교통안전표지(노면표시)는 도로교통법상...,자전거 횡단보도 내려서 끌고가기\n자전거 횡단보도와 자전거 표지를 보면 마치 자전거...,"[자전거, 횡단보도, 내리다, 끌다, 자전거, 횡단보도, 자전거, 표지, 보다, 마...","[자전거, 횡단보도, 내리, 어서, 끌, 고, 가, 기, 자전거, 횡단보도, 와, ..."
2,150597,범죄예방을 위한 투도어현관보조문 설치,2012-10-30,주택,- 주민을 위하여 노력하시는 직원여러분께 감사드립니다 - 아파트및 다세대 등 홀로 ...,"000님 안녕하세요? 000님께서 공동주택(아파트, 다세대, 연립)의 현관문에 1인...",범죄예방을 위한 투도어현관보조문 설치\n- 주민을 위하여 노력하시는 직원여러분께 감...,"[범죄, 예방, 위하다, 투, 도어, 현관, 보조, 문, 설치, 주민, 위하다, 노...","[범죄, 예방, 위하, ㄴ, 투, 도어, 현관, 보조, 문, 설치, 주민, 위하, ..."
3,150659,각각 집마다 간이소화기를 만들어 배치하는 것을 의무로 하였으면 좋겠습니다.,2012-10-31,안전,어제 어린 남매 두명이 집에 전자렌지로 인해 불이나서중태에 빠졌다는 기사를 보았습니...,기시행중임 -소방시설설치유지 및 안전관리에 관한 법률 제8조제1항 및 서울특별시 주...,각각 집마다 간이소화기를 만들어 배치하는 것을 의무로 하였으면 좋겠습니다.\n어제 ...,"[각각, 집, 간이, 소화기, 만들다, 배치, 의무, 좋다, 어제, 어리다, 남매,...","[각각, 집, 마다, 간이, 소화기, 를, 만들, 어, 배치, 하, 는, 것, 의무..."
4,150660,예식장내 부대시설인 음식점의 횡포를...,2012-10-31,경제,자녀의 결혼등으로 인한 예식이나 회갑 칠순등으로 인하여 웨딩홀이나 호텔등을 이용시에...,안녕하십니까? 우리시정에 큰 관심을 보내주신 데 대하여 깊은 감사드립니다. dski...,예식장내 부대시설인 음식점의 횡포를...\n자녀의 결혼등으로 인한 예식이나 회갑 칠...,"[예식장, 부대시설, 음식점, 횡포, 자녀, 결혼, 인하다, 예식, 회갑, 인하다,...","[예식장, 내, 부대시설, ㄴ, 음식점, 의, 횡포, 를, 자녀, 의, 결혼, 등,..."
...,...,...,...,...,...,...,...,...,...
9862,179106,지하철 임산부 지정석 관련,2017-08-31,교통,안녕하세요 수고하십니다.다름이 아니오라 지하철 임산부지정석 관련 건의 드립니다.현...,김혁님 안녕하십니까? 제안 참여에 감사드리며 “지하철 임산부 지정석”건에 대하여 답...,지하철 임산부 지정석 관련\n안녕하세요 수고하십니다.다름이 아니오라 지하철 임산부...,"[지하철, 임산부, 지정석, 다르다, 지하철, 임산부, 지정석, 건의, 드리다, 현...","[지하철, 임산부, 지정석, 하, 시, 어요, 하, 시, ㅂ니다, 다르, ㅁ, 아니..."
9863,179107,채동선 작곡가의 서울 집을 보존해 주세요.,2017-08-31,문화,서울 성북동의 옛날짜장면집 뒷 골목에 한옥집들 사이로 고풍스러운 2층 적산가옥이 보...,NaN,채동선 작곡가의 서울 집을 보존해 주세요.\n서울 성북동의 옛날짜장면집 뒷 골목에 ...,"[동선, 작곡가, 집, 보존, 주세, 요, 성북동, 옛날, 짜장면, 집, 뒷, 골목...","[채, 동선, 작곡가, 의, 집, 보존, 하, 아, 주세, 요, 성북동, 의, 옛날..."
9864,179108,한강및 지천에서의 안전,2017-08-31,환경,시민의 안전과 편의를 위하여 불철주야 수고 하심에 감사드립니다.저는 주로 한강및 한...,NaN,한강및 지천에서의 안전\n시민의 안전과 편의를 위하여 불철주야 수고 하심에 감사드립...,"[한강, 지천, 안전, 시민, 안전, 편의, 위하다, 불철주야, 하심, 드리다, 주...","[한강, 및, 지천, 에서, 의, 안전, 시민, 의, 안전, 과, 편의, 를, 위하..."
9865,179109,세빛섬 이전 가능한가 ?,2017-08-31,환경,안녕하세요 ?최근에 노들섬이 2018년 말 경 개발된다는 기사를 본것 같습니다.예전...,NaN,세빛섬 이전 가능한가 ?\n안녕하세요 ?최근에 노들섬이 2018년 말 경 개발된다는...,"[세빛섬, 이전, 노들섬, 말, 경, 개발, 기사, 보다, 오페라, 하우스, 짓다,...","[세빛섬, 이전, 가능, 하, ㄴ가, 하, 시, 어요, 에, 노들섬, 2018, 년..."


# 02 테스트 데이터 #

## 민주주의 서울 자유제안 ##
### 천만상상오아시스 학습데이터와 달리 민원 분류 컬럼이 없음 ###
### 학습데이터를 활용 해 테스트 데이터에 민원 분류를 자동으로 진행하고자 함 ###

In [27]:
# 민주주의 서울 제유제안 정보 CSV 파일의 경우 pandas에서 로드할때 깨지는 문제 발생. Excel 파일로 진행
test_df = pd.read_excel('민주주의 서울 자유제안 정보.xlsx')

In [28]:
test_df

,제안번호,제안제목,제안내용,득표,제안등록일자
0,186203,중금속 문제가 심각해지고 있는데 중금속으로 분리할수 있는 방법은 없을까요?,"<span style=""font-size: 12px;"">중금속 문제가 심각해지고 있...",0,2019-12-18 01:21:40.0
1,186202,제설제에 사용되는 cacl2에서 염소가 환경오염을 일으킨다는데 친환경적인걸로 바꿔주새요,제설제에 사용되는 cacl2에서 염소가 환경오염을 일으킨다는데 친환경적인걸로 바꿔주...,0,2019-12-18 00:41:00.0
2,186201,"지하철역 위의 임대아파트 건설 후 청년, 신혼부분에게 임대",서울시의 경우 지하철이 상당히 발달되어 있어 교통서비스를 제공하고 있습니다.&nbs...,0,2019-12-17 13:41:34.0
3,186200,재활용 정거장이나 원룸에 재활용 쓰레기통 지원을 늘려주세요,몇몇구에서는 재활용 쓰레기 분리수거함을 원룸촌에 제공해주는 것으로 알고있는데요. 대...,0,2019-12-17 11:10:03.0
4,186199,임산부석을 마냥 비워두지 않는 방법,<p>저는 지하철을 자주 애용하는 여성입니다.</p><p>임산부석이 지정되어 있어 ...,0,2019-12-17 10:55:54.0
...,...,...,...,...,...
4319,180039,버스 정류장 근처의 빛 공해를 해결해 주세요 !,&nbsp;강동구 5호선 명일역 3번 출구에서 올라오면&nbsp; &nbsp;바로&...,0,2017-12-22 16:37:33.0
4320,180038,서울시내버스 안내방송을 부탁드려요 !,&nbsp;서울 시내 버스를 자주 이용하는 편입니다.&nbsp; 버스 내부에는 주로...,1,2017-12-21 18:44:46.0
4321,180037,서울역 인근 버스정류장 명 병행표기 제안.,<p>안녕하세요.</p><p>버스를 타고 서울역을 자주 지나는데 항상 이에 의문을 ...,1,2017-12-21 11:13:14.0
4322,180036,따릉이 보관소 설치 제안,서초역 주변에 따릉이 보관소 설치를 제안합니다!,0,2017-12-20 18:54:23.0


## 데이터 전처리 ##

### 데이터 핸들링 ###

In [29]:
# '제안내용' 빈칸인 행 삭제
test_df = test_df.dropna(subset=['제안내용'])

In [30]:
# '제안내용' 중복 행 삭제
test_df = test_df.drop_duplicates(subset=['제안내용'])

In [31]:
# '제안제목' + '제안내용' = contents 컬럼
test_df['contents'] = test_df.apply(lambda x:x['제안제목']+'\n'+x['제안내용'],axis=1)
test_df['contents']

0       중금속 문제가 심각해지고 있는데 중금속으로 분리할수 있는 방법은 없을까요?\n<sp...
1       제설제에 사용되는 cacl2에서 염소가 환경오염을 일으킨다는데 친환경적인걸로 바꿔주...
2       지하철역 위의 임대아파트 건설 후 청년, 신혼부분에게 임대\n서울시의 경우 지하철이...
3       재활용 정거장이나 원룸에 재활용 쓰레기통 지원을 늘려주세요\n몇몇구에서는 재활용 쓰...
4       임산부석을 마냥 비워두지 않는 방법\n<p>저는 지하철을 자주 애용하는 여성입니다....
                              ...                        
4319    버스 정류장 근처의 빛  공해를   해결해 주세요  !\n&nbsp;강동구 5호선 ...
4320    서울시내버스 안내방송을 부탁드려요 !\n&nbsp;서울 시내 버스를 자주 이용하는 ...
4321    서울역 인근 버스정류장 명 병행표기 제안.\n<p>안녕하세요.</p><p>버스를 타...
4322            따릉이 보관소 설치 제안\n서초역 주변에 따릉이 보관소 설치를 제안합니다!
4323    이륜차(오토바이)업소 주변 행정지도 통해 거리질서 확립필요\n<p>서울시내 어디를 ...
Name: contents, Length: 4192, dtype: object

In [32]:
# contents 컬럼 중복 행 삭제
test_df = test_df.drop_duplicates('contents')

In [33]:
# '제안등록일자' 컬럼 시계열 데이터 타입으로 변경
test_df['제안등록일자'] = pd.to_datetime(test_df['제안등록일자'])
test_df['제안등록일자']

0      2019-12-18 01:21:40
1      2019-12-18 00:41:00
2      2019-12-17 13:41:34
3      2019-12-17 11:10:03
4      2019-12-17 10:55:54
               ...        
4319   2017-12-22 16:37:33
4320   2017-12-21 18:44:46
4321   2017-12-21 11:13:14
4322   2017-12-20 18:54:23
4323   2017-12-18 20:33:46
Name: 제안등록일자, Length: 4192, dtype: datetime64[ns]

### 텍스트 전처리 ###

In [34]:
#html tag 內 한글 제거
def remove_tag(content):
    cleaner = re.compile('<.*?>')
    cleantext = re.sub(cleaner,'',content)
    return cleantext

In [35]:
# 태그 패턴
tag_pattern = regex.compile(r'class\=\"*\p{Hangul}+\"*|font-family\: \"*\p{Hangul}+\"*|HY중고딕|서울남산체|맑은 고딕|함초롬바탕|굴림|굴림체|새굴림|고딕|나눔고딕|산돌고딕|모던고딕|한양중고딕|HY견명조|바탕|태그래픽|궁서|궁서체|휴먼 명조|휴먼명조|\&nbsp\;')

In [36]:
# 여러 html 태그 패턴 삭제
test_df['contents'] = test_df['contents'].map(lambda x:regex.sub(tag_pattern,'',remove_tag(x)))

In [37]:
test_df['contents']

0       중금속 문제가 심각해지고 있는데 중금속으로 분리할수 있는 방법은 없을까요?\n중금속...
1       제설제에 사용되는 cacl2에서 염소가 환경오염을 일으킨다는데 친환경적인걸로 바꿔주...
2       지하철역 위의 임대아파트 건설 후 청년, 신혼부분에게 임대\n서울시의 경우 지하철이...
3       재활용 정거장이나 원룸에 재활용 쓰레기통 지원을 늘려주세요\n몇몇구에서는 재활용 쓰...
4       임산부석을 마냥 비워두지 않는 방법\n저는 지하철을 자주 애용하는 여성입니다.임산부...
                              ...                        
4319    버스 정류장 근처의 빛  공해를   해결해 주세요  !\n강동구 5호선 명일역 3번...
4320    서울시내버스 안내방송을 부탁드려요 !\n서울 시내 버스를 자주 이용하는 편입니다. ...
4321    서울역 인근 버스정류장 명 병행표기 제안.\n안녕하세요.버스를 타고 서울역을 자주 ...
4322            따릉이 보관소 설치 제안\n서초역 주변에 따릉이 보관소 설치를 제안합니다!
4323    이륜차(오토바이)업소 주변 행정지도 통해 거리질서 확립필요\n서울시내 어디를 가보든...
Name: contents, Length: 4192, dtype: object

## 형태소 분석 ##

### 토크나이징 ###

In [38]:
# 특정 품사 토크나이징
test_df['tokens'] = test_df['contents'].progress_map(lambda x:morp(x))

100%|██████████| 4192/4192 [06:34<00:00, 10.63it/s]


In [39]:
# 모든 품사 토크나이징
test_df['all_tokens'] = test_df['contents'].progress_map(lambda x:komoran.get_morphes_by_tags(cleanText(x)))

100%|██████████| 4192/4192 [01:53<00:00, 37.03it/s]


### 불용어(Stopwords) 제거 ###

In [40]:
test_df['tokens'] = test_df['tokens'].map(lambda x:[w for w in x if not w in stopwords_set])
test_df['all_tokens'] = test_df['all_tokens'].map(lambda x:[w for w in x if not w in stopwords_set])

In [42]:
# 토크나이징 후에도 중복된 행 제거
test_df["res"] = test_df["tokens"].astype(str) #list to string
test_df = test_df.drop_duplicates(subset="res")
test_df = test_df.drop('res',1)
test_df = test_df.reset_index(drop=True) # index 리셋

In [43]:
test_df

,제안번호,제안제목,제안내용,득표,제안등록일자,contents,tokens,all_tokens
0,186203,중금속 문제가 심각해지고 있는데 중금속으로 분리할수 있는 방법은 없을까요?,"<span style=""font-size: 12px;"">중금속 문제가 심각해지고 있...",0,2019-12-18 01:21:40,중금속 문제가 심각해지고 있는데 중금속으로 분리할수 있는 방법은 없을까요?\n중금속...,"[중금속, 중금속, 분리, 중금속, 중금속, 분리, 폐수, 속, 강물, 속, 수돗물...","[중금속, 가, 심각, 하, 아, 지, 고, 있, 는데, 중금속, 으로, 분리, 하..."
1,186202,제설제에 사용되는 cacl2에서 염소가 환경오염을 일으킨다는데 친환경적인걸로 바꿔주새요,제설제에 사용되는 cacl2에서 염소가 환경오염을 일으킨다는데 친환경적인걸로 바꿔주...,0,2019-12-18 00:41:00,제설제에 사용되는 cacl2에서 염소가 환경오염을 일으킨다는데 친환경적인걸로 바꿔주...,"[제설, 염소, 환경오염, 일으키다, 환경, 바꾸다, 제설, 염소, 환경오염, 일으...","[제설, 제, 에, 되, 는, cacl, 2, 에서, 염소, 가, 환경오염, 일으키..."
2,186201,"지하철역 위의 임대아파트 건설 후 청년, 신혼부분에게 임대",서울시의 경우 지하철이 상당히 발달되어 있어 교통서비스를 제공하고 있습니다.&nbs...,0,2019-12-17 13:41:34,"지하철역 위의 임대아파트 건설 후 청년, 신혼부분에게 임대\n서울시의 경우 지하철이...","[지하철역, 위, 임대아파트, 건설, 후, 청년, 신혼, 임대, 지하철, 상당히, ...","[지하철역, 위, 의, 임대아파트, 건설, 후, 청년, 신혼, 에게, 임대, 의, ..."
3,186200,재활용 정거장이나 원룸에 재활용 쓰레기통 지원을 늘려주세요,몇몇구에서는 재활용 쓰레기 분리수거함을 원룸촌에 제공해주는 것으로 알고있는데요. 대...,0,2019-12-17 11:10:03,재활용 정거장이나 원룸에 재활용 쓰레기통 지원을 늘려주세요\n몇몇구에서는 재활용 쓰...,"[정거장, 원룸, 쓰레기통, 지원, 늘리다, 주세, 요, 몇몇, 구, 쓰레기, 분리...","[재, 정거장, 이나, 원룸, 에, 재, 쓰레기통, 지원, 늘리, 어, 주세, 요,..."
4,186199,임산부석을 마냥 비워두지 않는 방법,<p>저는 지하철을 자주 애용하는 여성입니다.</p><p>임산부석이 지정되어 있어 ...,0,2019-12-17 10:55:54,임산부석을 마냥 비워두지 않는 방법\n저는 지하철을 자주 애용하는 여성입니다.임산부...,"[임산부, 석, 마냥, 비우다, 저, 지하철, 자주, 애용, 여성, 임산부, 석, ...","[임산부, 석, 마냥, 비우, 어, 두, 지, 않, 는, 저, 는, 지하철, 자주,..."
...,...,...,...,...,...,...,...,...
4155,180039,버스 정류장 근처의 빛 공해를 해결해 주세요 !,&nbsp;강동구 5호선 명일역 3번 출구에서 올라오면&nbsp; &nbsp;바로&...,0,2017-12-22 16:37:33,버스 정류장 근처의 빛 공해를 해결해 주세요 !\n강동구 5호선 명일역 3번...,"[버스, 정류장, 근처, 빛, 공해, 해결, 강동구, 호선, 일, 역, 출구, 올라...","[버스, 정류장, 근처, 의, 빛, 공해, 를, 해결, 하, 아, 주, 시, 어요,..."
4156,180038,서울시내버스 안내방송을 부탁드려요 !,&nbsp;서울 시내 버스를 자주 이용하는 편입니다.&nbsp; 버스 내부에는 주로...,1,2017-12-21 18:44:46,서울시내버스 안내방송을 부탁드려요 !\n서울 시내 버스를 자주 이용하는 편입니다. ...,"[서울시내, 버스, 안내, 방송, 드리다, 시내, 버스, 자주, 버스, 내부, 주로...","[서울시내, 버스, 안내, 방송, 드리, 어요, 시내, 버스, 를, 자주, 하, 는..."
4157,180037,서울역 인근 버스정류장 명 병행표기 제안.,<p>안녕하세요.</p><p>버스를 타고 서울역을 자주 지나는데 항상 이에 의문을 ...,1,2017-12-21 11:13:14,서울역 인근 버스정류장 명 병행표기 제안.\n안녕하세요.버스를 타고 서울역을 자주 ...,"[서울역, 인근, 버스정류장, 병행, 표기, 버스, 타다, 서울역, 자주, 지나다,...","[서울역, 인근, 버스정류장, 명, 병행, 표기, 하, 시, 어요, 버스, 를, 타..."
4158,180036,따릉이 보관소 설치 제안,서초역 주변에 따릉이 보관소 설치를 제안합니다!,0,2017-12-20 18:54:23,따릉이 보관소 설치 제안\n서초역 주변에 따릉이 보관소 설치를 제안합니다!,"[따릉이, 보관소, 설치, 서초, 역, 따릉이, 보관소, 설치]","[따릉이, 보관소, 설치, 서초, 역, 에, 따릉이, 보관소, 설치, 를, 하, ㅂ니다]"
